# Deformation components of 3DGS
- conda env: 3dgs+3dsrf

## load output of 3DGS (class GaussianProp)
- path to `../../gaussian_splatting/output/f336a291-bnotALLcam`

In [ ]:
import os, sys
import h5py
from scene import Scene, GaussianModel
import numpy as np
from utils.sh_utils import clamp_sh2rgb

path_to_3dsrf = os.path.join(os.getcwd(), os.pardir, "3DSSL-WS23_IntuitiveAnimation")
assert os.path.exists(path_to_3dsrf)
sys.path.append(path_to_3dsrf)
from utils.gaussian_utils import GaussianProp
from utils.Dataset_handler import Filehandler
from utils.pickel_io import load_from_memory
from utils.OBJ_helper import OBJ

In [2]:
# [TODO] set the path to output
path_to_output = os.path.join("/mnt/hdd/----")
assert os.path.exists(path_to_output)

# [TODO] set the session id
session = "f336a291-bnotALLcam"
path_to_session = os.path.join(path_to_output, session)
assert os.path.exists(path_to_session)

In [ ]:
list_expNames, list_expPaths = Filehandler.dirwalker_InFolder(path_to_folder=path_to_session, prefix="E0")
print("the number of expressions: ", len(list_expNames))
max_iter = 10_000

GPs=[]

for i, (expName, expPath) in enumerate(zip(list_expNames, list_expPaths)):
    list_frames, list_framePaths = Filehandler.dirwalker_InFolder(path_to_folder=expPath, prefix="")
    # print(list_frames)
    for frame, frame_path in zip(list_frames, list_framePaths):
        path_to_PC = os.path.join(frame_path, "point_cloud")
        path_to_maxIter = os.path.join(path_to_PC, "iteration_"+str(max_iter))
        assert os.path.exists(path_to_PC) and os.path.exists(path_to_maxIter)
        gp = load_from_memory(path_to_memory=os.path.join(path_to_maxIter, "memory"), pickle_fname="gaussian_prop.pkl")
        # print(gp)
        GPs.append(gp)

Gaussian properties


                @dataclass
                class GaussianProp:
                        xyz: np.ndarray
                        normals: np.ndarray
                        f_dc: np.ndarray #this is SH_coeffs, needs to be converted to RGB by SH2RGB
                        f_rest: np.ndarray
                        opacities: np.ndarray
                        scale: np.ndarray
                        rotation: np.ndarray
                        covariance: np.ndarray

In [ ]:
def saveHDF5_GaussianProp(GPs, name_hdf5):
    xyzs = []
    normals = []
    rgbs = []
    f_dcs = []
    f_rest = []
    opacities = []
    scales = []
    rotations = []

    # verts 87652
    for i, gp in enumerate(GPs):
        xyzs.append(gp.xyz.flatten())
        normals.append(gp.normals.flatten())
        rgbs.append(clamp_sh2rgb(gp.f_dc).flatten())
        f_dcs.append(gp.f_dc.flatten())
        f_rest.append(gp.f_rest.flatten())
        opacities.append(gp.opacities.flatten())
        scales.append(gp.scale.flatten())
        rotations.append(gp.rotation.flatten())


    path_to_dump = os.path.join(os.getcwd(), "output", session)
    assert os.path.exists(path_to_dump)
    with h5py.File(os.path.join(path_to_dump, name_hdf5), "w") as f:
        dset = f.create_dataset(name = "xyz", data = xyzs)
        dset = f.create_dataset(name="normal", data = normals)
        dset = f.create_dataset(name = "rgb", data = rgbs)
        dset = f.create_dataset(name = "f_dc", data = f_dcs)
        dset = f.create_dataset(name = "f_rest", data = f_rest)
        dset = f.create_dataset(name = "opacity", data=opacities)
        dset = f.create_dataset(name = "scale", data = scales)
        dset = f.create_dataset(name = "rotation", data = rotations)
    
    print(f"save hdf5 at {os.path.join(path_to_dump, name_hdf5)}")

    xyzs = np.asarray(xyzs)
    print(f"xyz shape: {xyzs.shape}")
    normals = np.asarray(normals)
    print(f"normals shape: {normals.shape}")
    rgbs = np.asarray(rgbs)
    print(f"rgbs shape: {rgbs.shape}")
    f_dcs = np.asarray(f_dcs)
    print(f"f_dcs shape: {f_dcs.shape}")
    f_rest = np.asarray(f_rest)
    print(f"f_rest shape: {f_rest.shape}")
    opacities = np.asarray(opacities)
    print(f"opacities shape: {opacities.shape}")
    scales = np.asarray(scales)
    print(f"scales shape: {scales.shape}")
    rotations = np.asarray(rotations)
    print(f"rotation shape: {rotations.shape}")

# Save GaussianProp in .hdf5

In [ ]:
saveHDF5_GaussianProp(GPs=GPs, name_hdf5=session+"_datamat_87652.hdf5")

# Downsampling trained 3D Gaussians

The number of Gaussians = 5509
- This is equivalent to 2x de-subdivision

In [ ]:
saveHDF5_GaussianProp(GPs=GPs, name_hdf5=session+"_datamat_5509.hdf5")

The number of Gaussians = 21954
- This is equivalent to 1x de-subdivision

In [ ]:
saveHDF5_GaussianProp(GPs=GPs, name_hdf5=session+"_datamat_21954.hdf5")